## Out Of Vocabulary Identifier
Identify from the training texts all words which are not recognized by the **Fasttest** word embedding model. The result is persisted into ../../data/staging_data/fasttext_outofvocab_words.txt


In [1]:
from tqdm import tqdm
import os, re, csv, math, codecs
import numpy as np

embeddings_index = {}
f = codecs.open('../../pretrained_models/fasttext/wiki.fr.vec', encoding='utf-8')
for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

1152466it [02:24, 7955.62it/s]


In [4]:
from keras.preprocessing.text import Tokenizer
import pandas as pd

def identifyOutofVocabulary(inputFileName, outputFileName):
    XTrain = pd.read_csv(inputFileName, sep=';')

    tokenizer = Tokenizer(num_words= 10000)
    tokenizer.fit_on_texts(XTrain['question'])

    outOfVocabWords = []
    for word, i in tokenizer.word_index.items():                
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is None and not re.search(r"[a-zA-Zéèàîôûï]", word) is None:
            outOfVocabWords.append(word)

    pd.DataFrame(outOfVocabWords).to_csv(outputFileName, index=None, header=None)

In [7]:
identifyOutofVocabulary(
    inputFileName= '../../data/staging_data/clean_input_train.csv',
    outputFileName= '../../data/staging_data/fasttext_outofvocab_words.txt')